In [1]:
from copy import copy

In [166]:
VAR_INDEX = 0
SUBSTITUDE_INDEX = 0

# Abstract class Formula
class Formula(): 
    def __init__(self, args, negated=False):
        self.negated = negated
        self.args = args 
        self.tree_args = []
        self.is_substitude = False
        self.substitude = None
    def __neg__(self): 
        new_object = copy(self)
        new_object.negated = not new_object.negated
        return new_object 
    def __add__(self, other): return Disjunction(self, other)
    def __mul__(self, other): return Conjunction(self, other)
    def __rshift__(self, other): return Implication(self, other)
    def __mod__(self, other): return Equivalence(self, other)
    def __lt__(self, other):        
        if type(self) == Var and type(other) == Var:
            return self.name < other.name
        
        self_flattenize = []
        other_flattenize = []
        flattenize(self.tree_args, self_flattenize)
        flattenize(other.tree_args, other_flattenize)
        
        if type(self) == Var and (type(other) == Conjunction or type(other) == Disjunction):
            if is_all_leaves(other):
                return self.name <= other.args[0].name
            return self.name <= other_flattenize[0]
        if  (type(self) == Conjunction or type(self) == Disjunction) and type(other) == Var:
            if is_all_leaves(self):
                return self.args[0].name < other.name
            return self_flattenize[0] < other.name
        if  (type(self) == Conjunction or type(self) == Disjunction) and (type(other) == Conjunction or type(other) == Disjunction):
            if is_all_leaves(self) and is_all_leaves(other):
                return self.args < other.args
            return self_flattenize < other_flattenize
        
        
# Abstract class Connective 
class Connective(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)
        

# Conjunction
class Conjunction(Connective):
    pass

# Disjunction
class Disjunction(Connective): 
    pass

# Implication
class Implication(Formula):
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Equivalence
class Equivalence(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Variable
class Var(Formula):
    def __init__(self, name):
        super().__init__(None)        
        global VAR_INDEX
        self.index = VAR_INDEX
        VAR_INDEX += 1
        self.name = name

# Substitute for CNFs        
class Substitute(Formula):
    def __init__(self):
        super().__init__(None)        
        global SUBSTITUDE_INDEX
        self.index = SUBSTITUDE_INDEX
        SUBSTITUDE_INDEX += 1
        self.name = 'r' + str(SUBSTITUDE_INDEX)        

In [295]:
def remove_imp_eq(formula):
    if type(formula) == Var or type(formula) == Substitute:
        return formula
    expr = None
    if type(formula) == Conjunction:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1])
    
    if type(formula) == Disjunction:
        expr = remove_imp_eq(formula.args[0]) + remove_imp_eq(formula.args[1])
    
    if type(formula) == Implication:
        expr = remove_imp_eq(-(formula.args[0])) + remove_imp_eq(formula.args[1])
    
    if type(formula) == Equivalence:
        expr = (remove_imp_eq(-formula.args[0]) + remove_imp_eq(formula.args[1])) * \
               (remove_imp_eq(formula.args[0]) + remove_imp_eq(-formula.args[1]))
#         expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1]) + \
#                remove_imp_eq(-(formula.args[0])) * remove_imp_eq(-(formula.args[1]))
    expr.negated = formula.negated
    return expr

def de_morgan(formula):
    if type(formula) == Var or type(formula) == Substitute:
        return formula
    if type(formula) == Disjunction :
        if formula.negated:
            return de_morgan(-formula.args[0]) * de_morgan(-formula.args[1])
        else:
            return de_morgan(formula.args[0]) + de_morgan(formula.args[1])
    if type(formula) == Conjunction:
        if formula.negated:
            return de_morgan(-formula.args[0]) + de_morgan(-formula.args[1])
        else:
            return de_morgan(formula.args[0]) * de_morgan(formula.args[1])

def flatten_list(l):
    flat_list = []
    for list_elem in l:
        if type(list_elem) == list:
            for item in list_elem:
                flat_list.append(item)
        else:
            flat_list.append(list_elem)
    return flat_list

def remove_associativity(formula):
    if type(formula) != Var and type(formula) != Substitute: 
        for i, arg in enumerate(formula.args):
            if (type(formula) == Disjunction and type(formula.args[i]) == Disjunction) or \
            (type(formula) == Conjunction and type(formula.args[i]) == Conjunction):
                formula.args[i] = formula.args[i].args
                formula.args = flatten_list(formula.args)
                remove_associativity(formula)  
        for i, arg in enumerate(formula.args):
            remove_associativity(arg)

def get_tree_args(formula):
    if type(formula) != Var and type(formula) != Substitute:
        for arg in formula.args:
            get_tree_args(arg)
            if type(arg) == Conjunction or type(arg) == Disjunction:
                for inner_arg in arg.args:
                    if type(inner_arg) == Var:
                        inner_arg.tree_args = inner_arg.name
                        arg.tree_args += [inner_arg.name]
                    else:
                        arg.tree_args += [inner_arg.tree_args]
    else:
        formula.tree_args = formula.name
                        
def get_root_tree(formula):
    for arg in formula.args:
        formula.tree_args.append(arg.tree_args)
        
def get_formula_tree(formula):
    get_tree_args(formula)
    get_root_tree(formula)

    
def clear_formula_tree(formula):
    formula.tree_args = []
    if type(formula) != Var and type(formula) != Substitute:
        for arg in formula.args:
            clear_formula_tree(arg)

def is_all_leaves(formula):
    for arg in formula.args:
        if type(arg) != Var and type(arg) != Substitute and not arg.is_substitude:
            return False
    return True

def flattenize(tree_args, result):
    for i, elem in enumerate(tree_args):
        if type(elem) != list:
            result += [elem]
        else:
            flattenize(elem, result)

def sorting(formula):
    if type(formula) != Var:
        for arg in formula.args:
            sorting(arg)
        if type(formula) == Conjunction or type(formula) == Disjunction:
            formula.args = sorted(formula.args)
            clear_formula_tree(f)
            get_formula_tree(f)                

In [251]:
def find_substitutes(formula, R):
    if type(formula) != Var:
        for arg in formula.args:
            find_substitutes(arg, R)
        if (type(formula) == Conjunction or type(formula) == Disjunction) and is_all_leaves(formula):
            formula.is_substitude = True
            
def replace_substitudes(formula):
    if type(formula) != Var:
        for i, arg in enumerate(formula.args):
            replace_substitudes(arg)
        for i, arg in enumerate(formula.args):
            if arg.is_substitude:
                sub = Substitute()
                R.append(sub % arg)
                formula.args[i] = sub

def replace_root_substitude(formula, R):
    if formula.is_substitude:
        sub = Substitute()
        R.append(sub % formula)
        return sub 
    
def CNF(formula, R):
    for r in R:
        formula *= r
    return formula

In [371]:
def distribute(formula):
    for i, disj in enumerate(formula.args):  
        if type(disj) != Substitute:
            if type(disj.args[1]) == Conjunction:
                new_arg = []
                print(disj.args[1])
                for conj in disj.args[1].args:
                    new_arg.append(disj.args[0] + conj)
                new_disj = new_arg[0] + disj.args[0]
                for j in range(1, len(new_arg)):
                    new_disj *= new_arg[i]  + disj.args[0]
                formula.args[i] = new_disj

In [372]:
r1 = Substitute()
r2 = Substitute()
f = r1 * (r2 + (a * b))

f.args

In [373]:
distribute(f)

In [374]:
get_formula_tree(f)

In [375]:
f.tree_args

['r91', [[['r92', 'a'], 'r92'], [['r92', 'b'], 'r92']]]

In [387]:
remove_associativity(f)

In [392]:
f.args[1].args

In [334]:
a = Var('a')
b = Var('b')
c = Var('c')
d = Var('d')
e = Var('e')
g = Var('g')
a,b,c,d,e

(<__main__.Var at 0x1757608e160>,
 <__main__.Var at 0x1757608e7f0>)

In [335]:
# f = (d+c+e*(d+a)*((g*g) + a)) * (a*g + c*c) * e * -(b + a + c) 
f = a + a + (b * (a+c) * d)
f = de_morgan(f)
remove_associativity(f)
get_formula_tree(f)
print(f.tree_args)

sorting(f)
print(f.tree_args)

R = []
find_substitutes(f, R)
replace_substitudes(f)
f = replace_root_substitude(f, R)
f = CNF(f, R)

# remove_associativity(f)
f.args

['a', 'a', ['b', ['a', 'c'], 'd']]
['a', 'a', [['a', 'c'], 'b', 'd']]


In [336]:
f = remove_imp_eq(f)
f = de_morgan(f)
remove_associativity(f)
clear_formula_tree(f)
get_formula_tree(f)

In [329]:
f.args

In [337]:
f.tree_args

['r74',
 ['r72', 'a', 'c'],
 ['r72', ['a', 'c']],
 ['r73', ['r72', 'b']],
 ['r73', 'r72', 'b'],
 ['r74', 'a', 'a'],
 ['r74', ['a', 'a']]]

In [ ]:
def DIMACS(cnf)